In [1]:
import os
import h5py
import numpy as np
from skimage.transform import resize

In [2]:
#The path to my training dataset
data_dir = 'C:\\Users\\VANSH GARG\\.cache\\kagglehub\\datasets\\awsaf49\\brats2020-training-data\\versions\\3\\BraTS2020_training_data\\content\\data'
h5_files = [f for f in sorted(os.listdir(data_dir)) if f.endswith('.h5')]

In [ ]:
'''The below function is a generalised function with extracts the data from the HDF5 files which exists in the form of 2D slices. The function then 
stacks the slices of the same volume in sequential order and then stores the new 4 Dimensional array into a new HDF5 file. I named brain_tumor_data4.h5
Along the way the function also preprocesses the data as is required.'''
def data_generation(files):
    data_sorted=[0]*(155*369)
    for i in files:
        L=i.split("_")
        k=int(L[-1].split(".")[0])
        n=int(L[1])
        data_sorted[(155*(n-1))+k]=i
    #Data_sorted contains proper sorted order of the slices but in the form of a list 
    img_slices=[]
    msk_slices=[]
    #Created two slices to store the 3 layer(depth) in it
    j=0
    with h5py.File("brain_tumor_data4.h5", "w") as hf:
        #Created two HDF5 data files which is 5 dimensional in nature and stores value in float16 format
        hf.create_dataset("X", shape=(369, 128, 128, 32, 4), dtype='float16', chunks=True)
        hf.create_dataset("Y", shape=(369, 128, 128, 32), dtype='float16', chunks=True)
        for k,i in enumerate(data_sorted):
            j+=1
            file=os.path.join(data_dir,i)
            with h5py.File(file, 'r') as f:
                #Normalising and reducing the size of image and mask data files
                image_data=np.array(f['image'],dtype='float16')
                image_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data) + 1e-6)
                mask_data=np.array(f['mask'],dtype='float16')
                mask_data = np.mean(mask_data, axis=-1) #Taking the mean of the mask file along it's channels
                mask_data = (mask_data - np.min(mask_data)) / (np.max(mask_data) - np.min(mask_data) + 1e-6)
                if np.max(mask_data) > 0: #This condition allows only data files with a tumor region in it to be appeneded. This ensures that as much information is retained as possible. To properly train the model.
                    img_slices.append(image_data)
                    msk_slices.append(mask_data)
                if j==155:
                    img_voxel=np.stack(img_slices,axis=2)  #Stacking the image and mask slices into the 3 axis of image voxels
                    msk_voxel=np.stack(msk_slices,axis=2)  
                    img_voxel= resize(img_voxel,(128,128,32,4),preserve_range=True,anti_aliasing=True) #Resizing the voxels into a uniform size to ensure uniformity. Used features such as anti_aliasing and preserve_range.
                    msk_voxel= resize(msk_voxel,(128,128,32),preserve_range=True,anti_aliasing=True)
                    hf["X"][(k//155)] = img_voxel #This line makes sure to properly index my data and make it sequential.
                    hf["Y"][(k//155)] = msk_voxel
                    img_slices=[]
                    msk_slices=[]
                    j=0
                    continue    
data_generation(h5_files)
